In [1]:
from fypy.model.levy.TemperedStable import TemperedStable
from fypy.model.levy.VarianceGamma import VarianceGamma
from fypy.model.levy.BlackScholes import BlackScholes
from fypy.model.levy.NIG import NIG


from fypy.pricing.fourier.ProjEuropeanPricer import ProjEuropeanPricer
from fypy.pricing.fourier.LewisEuropeanPricer import LewisEuropeanPricer
from fypy.pricing.fourier.CarrMadanEuropeanPricer import CarrMadanEuropeanPricer
from fypy.pricing.fourier.GilPeleazEuropeanPricer import GilPeleazEuropeanPricer
from fypy.pricing.fourier.HilbertEuropeanPricer import HilbertEuropeanPricer


from fypy.termstructures.DiscountCurve import DiscountCurve_ConstRate
from fypy.termstructures.EquityForward import EquityForward

from fypy.pricing.StrikesPricer import StrikesPricer

In [2]:
import time
import pandas as pd
import numpy as np

In [3]:
# option params
S0 = 100
K = 100
r = 0.05
q = 0.01
T = 2

In [4]:
# rate curves
disc_curve = DiscountCurve_ConstRate(rate=r)
div_disc = DiscountCurve_ConstRate(rate=q)
fwd = EquityForward(S0=S0, discount=disc_curve, divDiscount=div_disc)

In [5]:
ts_params = dict(
    alpha_p=1.2, beta_p=0.4, lambda_p=2.2, alpha_m=0.2, beta_m=0.6, lambda_m=1.3
)
# model = TemperedStable(forwardCurve=fwd, discountCurve=disc_curve, **ts_params)
model = VarianceGamma(
    forwardCurve=fwd, discountCurve=disc_curve, sigma=0.2, nu=0.85, theta=-0.1
)
# model = BlackScholes(forwardCurve=fwd, discountCurve=disc_curve, sigma=0.15)
# model = NIG(forwardCurve=fwd, discountCurve=disc_curve, alpha=8.9932, beta=-4.5176, delta = 1.1528)

In [6]:
# pricers
proj_pricer = ProjEuropeanPricer(model=model, N=2**8, order=3)
lewis_pricer = LewisEuropeanPricer(model=model, N=2**8)
carr_madan_pricer = CarrMadanEuropeanPricer(model=model, N=2**20)
gil_peleaz_pricer = GilPeleazEuropeanPricer(model=model)
hilbert_pricer = HilbertEuropeanPricer(model=model)


pricers = {
    "proj": proj_pricer,
    "lewis": lewis_pricer,
    "carr_madan": carr_madan_pricer,
    "gil_peleaz": gil_peleaz_pricer,
    "hilbert": gil_peleaz_pricer,

}

In [7]:
# carr_madan_pricer_vg = CarrMadanEuropeanPricer(model=model_vg, N=2**16, alpha=0.75)
# print(carr_madan_pricer_vg.price(T=T, K=K, is_call=True))
# carr_madan_pricer_bs = CarrMadanEuropeanPricer(model=model_bs, N=2**8, alpha=0.75)
# print(carr_madan_pricer_bs.price(T=T, K=K, is_call=True))
# carr_madan_pricer_nig = CarrMadanEuropeanPricer(model=model_nig, N=2**16)
# print(carr_madan_pricer_nig.price(T=T, K=K, is_call=True))

In [8]:
print(lewis_pricer.price(T=T, K=K, is_call=False))
print(proj_pricer.price(T=T, K=K, is_call=False))
print(carr_madan_pricer.price(T=T, K=K, is_call=False))
print(gil_peleaz_pricer.price(T=T, K=K, is_call=False))
print(hilbert_pricer.price(T=T, K=K, is_call=False))


7.585312173727928
7.585312162318126
7.585312184656464
7.585312173530383
7.585267313575384


In [75]:
def time_pricer(pricer: StrikesPricer, T: float, K: float, N_test: int = 2):
    t = time.time()
    for _ in range(N_test):
        price = pricer.price(T, K, is_call=True)
    t_end = time.time() - t
    return (t_end) / N_test


def get_computation_times(pricers_dict, T, K):
    computation_times = {}
    for pricer_name, pricer in pricers_dict.items():
        time = time_pricer(pricer, T, K)
        computation_times[pricer_name] = time
    return computation_times


def get_prices(pricers_dict, T, K):
    prices = {}
    for pricer_name, pricer in pricers_dict.items():
        prices[pricer_name] = pricer.price(T, K, is_call=True)
    return prices


print(get_computation_times(pricers, 0.1, 110))
print(get_prices(pricers, 0.1, 110))

{'proj': 0.0012314319610595703, 'lewis': 0.01289224624633789, 'gil_peleaz_pricer': 0.6954020261764526}
{'proj': 4.6162343978281, 'lewis': 4.616603457056257, 'gil_peleaz_pricer': 4.616701334278561}


# Short time behaviour

In [76]:
ttms = np.arange(1e-2, 1, 1e-2)


def get_short_time_prices(pricers_dict, ttms):
    prices = {}
    for pricer_name, pricer in pricers_dict.items():
        price_pricer = []
        for ttm in ttms:
            price = pricer.price(ttm, K, is_call=True)
            price_pricer.append(price)
        prices[pricer_name] = price_pricer
    return prices


get_short_time_prices(pricers, ttms)

{'proj': [0.671442298130023,
  1.259275649659316,
  1.8440585323092162,
  2.418665874608255,
  2.9785039640480115,
  3.5214956946348623,
  4.047260056523106,
  4.556430315800504,
  5.050172257100688,
  5.529866950318845,
  5.997059290385687,
  6.452761423524009,
  6.89828793695609,
  7.334636252871827,
  7.76264221629841,
  8.182995011903714,
  8.59625735168151,
  9.002886925185084,
  9.403256791000747,
  9.797673512554429,
  10.186392559685766,
  10.569630931786804,
  10.947577200570786,
  11.320399287640425,
  11.688250331256318,
  12.051272990396965,
  12.409602503692529,
  12.763435347319804,
  13.11276157025803,
  13.45777339685802,
  13.798590833492904,
  14.135331283875079,
  14.468109518448284,
  14.7970375379309,
  15.122224383853833,
  15.443775934216626,
  15.761794710696254,
  16.0763797146909,
  16.387626302412617,
  16.69562610389412,
  17.000466986823795,
  17.302233063313736,
  17.601004735799567,
  17.896858777102466,
  18.189868439066537,
  18.480103584011516,
  18.76